In [1]:
import numpy as np
import scs

ModuleNotFoundError: No module named 'scs'

In [2]:
#############################################
#      Generate random cone problems        #
#############################################

def pos(x):
    return (x + np.abs(x)) / 2.
    
def gen_feasible(m, n, p_scale = 0.1):
    P = np.random.randn(n,n)
    P = p_scale * P.T @ P

    # Make problem slightly more numerically challenging:
    A = np.random.randn(m, n)
        
    x = np.random.randn(n)
    c = -A.T @ y - P @ x
    b = np.random.randn(m)
        
    return (P, A, b, c)

In [6]:
m = 1500
n = 1000
N = int(5e3)
seed = 1234

In [1]:
np.random.seed(seed)
cone = dict(l=2, s=[3, 4], cs=[2, 3])
m = dict['l'] + sum([j * (j+1) / 2 for j in dict['s']) + sum([j * j for j in dict['cs'])
n = m // 2
(P, A, b, c) = gen_feasible(m, n)
probdata = dict(P=scipy.sparse.csc_matrix(P), A=scipy.sparse.csc_matrix(A), b=b, c=c)
sol = scs.solve(probdata, cone)

NameError: name 'np' is not defined

In [8]:
print(c.T @ sol['x'])
print(np.linalg.norm(c) * np.linalg.norm(A @ sol['x'] + sol['s']))
print(np.sqrt(max(0., sol['x'].T @ P @ sol['x'])))

-0.9999999999999991
0.00016302068657004883
6.998773443321813e-07


In [9]:
print(np.linalg.norm(P - P.T))

0.0


In [10]:
np.random.seed(seed)
(P, A, b, c) = gen_infeasible(m, n)
#P = P + np.eye(n)
(x,y) = solve_qp_dr(P, A, b, c, N=N)
probdata = dict(P=scipy.sparse.csc_matrix(P), A=scipy.sparse.csc_matrix(A), b=b, c=c)
cone = dict(l=m)
sol = scs.solve(probdata, cone, normalize=True, max_iters=N, acceleration_lookback = 10, eps=1e-6)

/Users/bodonoghue/miniconda2/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:56: RuntimeWarning: divide by zero encountered in true_divide
/Users/bodonoghue/miniconda2/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:57: RuntimeWarning: divide by zero encountered in true_divide
/Users/bodonoghue/miniconda2/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in matmul
  
/Users/bodonoghue/miniconda2/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in matmul
  after removing the cwd from sys.path.
/Users/bodonoghue/miniconda2/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in matmul
  if sys.path[0] == '':
/Users/bodonoghue/miniconda2/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in matmul
  from ipykernel import kerne

518
------------------------------------------------------------------
	       SCS v3.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 1000, constraints m: 1500
cones: 	  l: linear vars: 1500
settings: eps: 1.00e-06, alpha: 1.50, max_iters: 5000,
	  normalize: 1, scale: 1.00, adaptive_scaling: 1,
	  acceleration_lookback: 10, acceleration_interval: 50,
	  warm_start: 0
lin-sys:  sparse-direct
	  nnz(A): 1500000, nnz(P): 500500
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 2.75e-01  4.81e+00  2.28e+00 -2.51e+00  1.00e+00  2.30e-02 
   100| 7.54e-01  4.95e+00  4.02e-01  6.90e+02  1.00e+00  8.93e-01 
   200| 2.74e+13  1.92e+14  1.09e+19  2.42e+14  1.00e+00  1.75e+00 
   300| 7.04e+10  5.99e+11  1.09e+19  5.00e